# Graph Neural Network (GCN)-based Synthetic Binding Logic Classification plus Graph-SafeML
The eisting example of GNN-based Synthetic Binding Logic Classification from google research team is used to test the idea of SafeML for Graph-based classifiers. You can find the source code [here](https://github.com/google-research/graph-attribution) and the related paper for the code is available [here](https://papers.nips.cc/paper/2020/file/417fbbf2e9d5a28a855a11894b2e795a-Paper.pdf) [[1]](https://papers.nips.cc/paper/2020/file/417fbbf2e9d5a28a855a11894b2e795a-Paper.pdf).
Regarding the Graph-based distance measure, the theory of "Graph distance for complex networks" provided by of Yutaka Shimada et al. is used [[2]](https://www.nature.com/articles/srep34944). The code related to this paper is avaialble [here](https://github.com/msarrias/graph-distance-for-complex-networks).
You can read more about the idea of SafeML in [[3]](https://github.com/ISorokos/SafeML). To read more about "Synthetic Binding Logic Classification" and the related dataset that is used in this notebook, please check [[4]](https://www.pnas.org/content/pnas/116/24/11624.full.pdf).

![SafeML logo from: https://github.com/ISorokos/SafeML](https://miro.medium.com/max/700/1*H0lN2Q9lmSRgfaGj9VqqGA.png)

The SafeML project takes place at the University of Hull in collaboration with Fraunhofer IESE and Nuremberg Institute of Technology


## Table of Content
* [Initialization and Installations](#init)
* [Importing Required Libraries](#lib)
* [Graph Attribution Specific Imports](#glib)
* [Load Experiment Data, Task and Attribution Techniques](#load)
* [Creating a GNN Model](#model)
* [Graph Vizualization](#gviz)
* [Graph Distance Measures and SafeML Idea](#SafeML)
* [Discussion](#dis)

### References:
[[1]. Wiltschko, A. B., Sanchez-Lengeling, B., Lee, B., Reif, E., Wei, J., McCloskey, K. J., & Wang, Y. (2020). Evaluating Attribution for Graph Neural Networks.](https://papers.nips.cc/paper/2020/file/417fbbf2e9d5a28a855a11894b2e795a-Paper.pdf)

[[2]. Shimada, Y., Hirata, Y., Ikeguchi, T., & Aihara, K. (2016). Graph distance for complex networks. Scientific reports, 6(1), 1-6.](https://www.nature.com/articles/srep34944)

[[3]. Aslansefat, K., Sorokos, I., Whiting, D., Kolagari, R. T., & Papadopoulos, Y. (2020, September). SafeML: Safety Monitoring of Machine Learning Classifiers Through Statistical Difference Measures. In International Symposium on Model-Based Safety and Assessment (pp. 197-211). Springer, Cham.](https://arxiv.org/pdf/2005.13166.pdf)

[[4]. McCloskey, K., Taly, A., Monti, F., Brenner, M. P., & Colwell, L. J. (2019). Using attribution to decode binding mechanism in neural network models for chemistry. Proceedings of the National Academy of Sciences, 116(24), 11624-11629.](https://www.pnas.org/content/pnas/116/24/11624.full.pdf)

<a id = "init"></a>
## Initialization and Installations

In [ ]:
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import sys

IN_COLAB = 'google.colab' in sys.modules
REPO_DIR = '..' if IN_COLAB  else '..'

In [ ]:
!git clone https://github.com/google-research/graph-attribution.git --quiet
    
import sys
sys.path.insert(1, '/kaggle/working/graph-attribution')

In [ ]:
!pip install tensorflow tensorflow-probability -q
!pip install dm-sonnet -q
!pip install graph_nets "tensorflow>=2.1.0-rc1" "dm-sonnet>=2.0.0b0" tensorflow_probability
!pip install git+https://github.com/google-research/graph-attribution -quiet

In [ ]:
!pip install git+https://github.com/google-research/graph-attribution

<a id = "lib"></a>
## Importing Required Libraries

In [ ]:
import os
import itertools
import collections
import tqdm.auto as tqdm

from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
import sonnet as snt
import graph_nets
from graph_nets.graphs import GraphsTuple
import graph_attribution as gatt

from tqdm import tqdm

import time

import networkx as nx

# Ignore tf/graph_nets UserWarning:
# Converting sparse IndexedSlices to a dense Tensor of unknown shape
import warnings
warnings.simplefilter("ignore", UserWarning)

for mod in [tf, snt, gatt]:
    print(f'{mod.__name__:20s} = {mod.__version__}')

<a id = "glib"></a>
## Graph Attribution specific imports

In [ ]:
from graph_attribution import tasks
from graph_attribution import graphnet_models as gnn_models
from graph_attribution import graphnet_techniques as techniques
from graph_attribution import datasets
from graph_attribution import experiments
from graph_attribution import templates
from graph_attribution import graphs as graph_utils

#datasets.DATA_DIR = os.path.join(REPO_DIR, 'data')
#print(f'Reading data from: {datasets.DATA_DIR}')

datasets.DATA_DIR = './graph-attribution/data'

<a id = "load"></a>
# Load Experiment Data, Task and Attribution Techniques

In [ ]:
print(f'Available tasks: {[t.name for t in tasks.Task]}')
print(f'Available model types: {[m.name for m in gnn_models.BlockType]}')
print(f'Available ATT techniques: {list(techniques.get_techniques_dict(None,None).keys())}')

In [ ]:
task_type = 'logic7'
block_type = 'gcn'

#task_dir = datasets.get_task_dir(task_type)
task_dir = './graph-attribution/data/logic7'
exp, task, methods = experiments.get_experiment_setup(task_type, block_type)
task_act, task_loss = task.get_nn_activation_fn(), task.get_nn_loss_fn()
graph_utils.print_graphs_tuple(exp.x_train)
print(f'Experiment data fields:{list(exp.__dict__.keys())}')

<a id = "model"></a>
## Creating a GNN Model

### Defining Hyperparams of the Experiment

In [ ]:
hp = gatt.hparams.get_hparams({'block_type':block_type, 'task_type':task_type})
hp

### Instantiate model

In [ ]:
model = experiments.GNN(node_size = hp.node_size,
               edge_size = hp.edge_size,
               global_size = hp.global_size,
               y_output_size = task.n_outputs,
               block_type = gnn_models.BlockType(hp.block_type),
               activation = task_act,
               target_type = task.target_type,
               n_layers = hp.n_layers)
model(exp.x_train)
gnn_models.print_model(model)

<a id ="train"></a>
## Training the GNN Model

In [ ]:
optimizer = snt.optimizers.Adam(hp.learning_rate)

opt_one_epoch = gatt.training.make_tf_opt_epoch_fn(exp.x_train, exp.y_train, hp.batch_size, model,
                                      optimizer, task_loss)

pbar = tqdm(range(hp.epochs))
losses = collections.defaultdict(list)
start_time = time.time()
for _ in pbar:
    train_loss = opt_one_epoch(exp.x_train, exp.y_train).numpy()
    losses['train'].append(train_loss)
    losses['test'].append(task_loss(exp.y_test, model(exp.x_test)).numpy())
    #pbar.set_postfix({key: values[-1] for key, values in losses.items()})

losses = {key: np.array(values) for key, values in losses.items()}

In [ ]:
# Plot losses
for key, values in losses.items():
    plt.plot(values, label=key)
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend()
plt.show()

In [ ]:
y_pred = model(exp.x_test).numpy()
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0
#y_pred

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(accuracy_score(exp.y_test, y_pred))

print(confusion_matrix(exp.y_test, y_pred))

print(classification_report(exp.y_test, y_pred))

In [ ]:
# Evaluate predictions and attributions
results = []
for method in tqdm(methods.values(), total=len(methods)):
    results.append(experiments.generate_result(model, method, task, exp.x_test, exp.y_test, exp.att_test))
pd.DataFrame(results)

<a id = "gviz"></a>
## Graph Vizualization

In [ ]:
# Source: https://notebook.community/deepmind/graph_nets/graph_nets/demos/graph_nets_basics
graphs_nx = graph_nets.utils_np.graphs_tuple_to_networkxs(exp.x_test)

def nx_g_plotter(graphs_nx, ColNum=8, node_clr='#ff8080'):
    _, axs = plt.subplots(ncols=ColNum, nrows = 1, figsize=(30, 5))
    for iax, (graph_nx2, ax) in enumerate(zip(graphs_nx, axs)):
        nx.draw(graph_nx2, ax=ax, node_color=node_clr)
        ax.set_title("Graph {}".format(iax))

In [ ]:
graphs_nx_1 = []
graphs_nx_0 = []

for ii, g_net_ii in enumerate(graphs_nx):
    if exp.y_test[ii] == 1:
        graphs_nx_1.append(g_net_ii)
    else:
        graphs_nx_0.append(g_net_ii)
        
nx_g_plotter(graphs_nx_1, ColNum=8, node_clr='#ff8080')
nx_g_plotter(graphs_nx_0, ColNum=8, node_clr='#00bfff')

In [ ]:
graphs_nx_wrong0 = []
graphs_nx_wrong1 = []
graphs_nx_correct0 = []
graphs_nx_correct1 = []

y_pred2 = model(exp.x_test).numpy()

y_wrong0 = []
y_wrong1 = []
y_correct0 = []
y_correct1 = []

for ii, g_net_ii in enumerate(graphs_nx):
    if exp.y_test[ii] != y_pred[ii] and exp.y_test[ii] == 0:
        graphs_nx_wrong0.append(g_net_ii)
        y_wrong0.append(y_pred2[ii])
    elif exp.y_test[ii] != y_pred[ii] and exp.y_test[ii] == 1:
        graphs_nx_wrong1.append(g_net_ii)
        y_wrong1.append(y_pred2[ii])
    elif exp.y_test[ii] == y_pred[ii] and exp.y_test[ii] == 0:
        graphs_nx_correct0.append(g_net_ii)
        y_correct0.append(y_pred2[ii])
    elif exp.y_test[ii] == y_pred[ii] and exp.y_test[ii] == 1:
        graphs_nx_correct1.append(g_net_ii)
        y_correct1.append(y_pred2[ii])
        
print(len(graphs_nx_wrong0), len(graphs_nx_wrong1), len(graphs_nx_correct0), len(graphs_nx_correct1))

nx_g_plotter(graphs_nx_wrong0, ColNum=8, node_clr='#ff8080')
nx_g_plotter(graphs_nx_wrong1, ColNum=8, node_clr='#00bfff')
nx_g_plotter(graphs_nx_correct0, ColNum=8, node_clr='#00e600')
nx_g_plotter(graphs_nx_correct1, ColNum=8, node_clr='#e600ac')

In [ ]:
y_yes = exp.y_test[exp.y_test == 1]
y_no = exp.y_test[exp.y_test != 1]
y_yes.shape, y_no.shape

In [ ]:
recovered_data_dict_list = graph_nets.utils_np.graphs_tuple_to_data_dicts(exp.x_test)

graphs_tuple_1 = graph_nets.utils_np.data_dicts_to_graphs_tuple(recovered_data_dict_list)

<a id = "SafeML"></a>
## Graph Distance Measures and SafeML Idea

In [ ]:
!git clone https://github.com/msarrias/graph-distance-for-complex-networks --quiet

import sys
sys.path.insert(1, '/kaggle/working/graph-distance-for-complex-networks')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
import scipy.linalg as la
import networkx as nx
import random, time, math
from collections import Counter

import fun as f
from Graph import Graph
from Watts_Strogatz import watts_strogatz_graph
from Erdos_Renyi import erdos_renyi_graph

def Wasserstein_Dist(cdfX, cdfY):
  
    Res = 0
    power = 1
    n = len(cdfX)

    for ii in range(0, n-2):
        height = abs(cdfX[ii]-cdfY[ii])
        width = cdfX[ii+1] - cdfX[ii]
        Res = Res + (height ** power) * width 
 
    return Res


def r_eigenv(G_i, G_j):
    #Eigen-decomposition of G_j
    A_Gi = (nx.adjacency_matrix(G_i)).todense()
    D_i = np.diag(np.asarray(sum(A_Gi))[0])
    eigenvalues_Gi, eigenvectors_Gi = la.eig(D_i - A_Gi)
    r_eigenv_Gi = sorted(zip(eigenvalues_Gi.real, eigenvectors_Gi.T), key=lambda x: x[0])

    #Eigen-decomposition of G_j
    A_Gj = (nx.adjacency_matrix(G_j)).todense()
    D_j = np.diag(np.asarray(sum(A_Gj))[0])
    eigenvalues_Gj, eigenvectors_Gj = la.eig(D_j - A_Gj)
    r_eigenv_Gj = sorted(zip(eigenvalues_Gj.real, eigenvectors_Gj.T), key=lambda x: x[0])
    
    r = 4
    signs =[-1,1]
    temp = []
    for  sign_s in signs:
        for sign_l in signs:
            vri = sorted(f.normalize_eigenv(sign_s * r_eigenv_Gi[r][1]))
            vrj = sorted(f.normalize_eigenv(sign_l * r_eigenv_Gj[r][1]))
            cdf_dist = f.cdf_dist(vri, vrj)
            temp.append(cdf_dist)
    
    #Compute empirical CDF
    step = 0.005
    x=np.arange(0, 1, step)
    cdf_grid_Gip = f.cdf(len(r_eigenv_Gi[r][1]),x,
                   f.normalize_eigenv(sorted(r_eigenv_Gi[r][1], key=lambda x: x)))
    cdf_grid_Gin = f.cdf(len(r_eigenv_Gi[r][1]),x,
                   f.normalize_eigenv(sorted(-r_eigenv_Gi[r][1], key=lambda x: x)))

    cdf_grid_Gjp = f.cdf(len(r_eigenv_Gj[r][1]),x,
                   f.normalize_eigenv(sorted(r_eigenv_Gj[r][1], key=lambda x: x)))
    cdf_grid_Gjn = f.cdf(len(r_eigenv_Gj[r][1]),x,
                   f.normalize_eigenv(sorted(-r_eigenv_Gj[r][1], key=lambda x: x)))
    
    WD1 = Wasserstein_Dist(cdf_grid_Gip, cdf_grid_Gjp)
    WD2 = Wasserstein_Dist(cdf_grid_Gip, cdf_grid_Gjn)
    WD3 = Wasserstein_Dist(cdf_grid_Gin, cdf_grid_Gjp)
    WD4 = Wasserstein_Dist(cdf_grid_Gin, cdf_grid_Gjn)

    WD = [WD1, WD2, WD3, WD4]
    
    return max(temp), max(WD)

distt_wrong1_correct1 = np.zeros((len(graphs_nx_wrong1),len(graphs_nx_correct1)))
WDist_wrong1_correct1 = np.zeros((len(graphs_nx_wrong1),len(graphs_nx_correct1)))
Conf_W1_C1 = np.zeros((len(graphs_nx_wrong1),len(graphs_nx_correct1)))

for ii, g_net_ii in enumerate(graphs_nx_wrong1):
    for jj, g_net_jj in enumerate(graphs_nx_correct1):
        distt_wrong1_correct1[ii,jj], WDist_wrong1_correct1[ii,jj] = r_eigenv(g_net_ii, g_net_jj)
        Conf_W1_C1[ii,jj] = y_correct1[jj] - y_wrong1[ii]


import seaborn as sns; sns.set_theme()

#ax = sns.heatmap(distt)
#ax = sns.displot(distt_wrong1_correct1.flatten())

In [ ]:
df = pd.DataFrame()
df['WDist_W1_C1'] = WDist_wrong1_correct1.flatten()
df['Conf_W1_C1'] = Conf_W1_C1.flatten()

sns.scatterplot(data=df, x="Conf_W1_C1", y="WDist_W1_C1")

In [ ]:
graphs_nx_train = graph_nets.utils_np.graphs_tuple_to_networkxs(exp.x_train)

graphs_nx_train_1 = []
graphs_nx_train_0 = []

for ii, g_net_ii in enumerate(graphs_nx_train):
    if exp.y_train[ii] == 1:
        graphs_nx_train_1.append(g_net_ii)
    else:
        graphs_nx_train_0.append(g_net_ii)


distt_wrong1_train1 = np.zeros((len(graphs_nx_wrong1),len(graphs_nx_train_1)))
WDist_wrong1_train1 = np.zeros((len(graphs_nx_wrong1),len(graphs_nx_train_1)))

for ii, g_net_ii in enumerate(graphs_nx_wrong1):
    for jj, g_net_jj in enumerate(graphs_nx_train_1):
        distt_wrong1_train1[ii,jj], WDist_wrong1_train1[ii,jj] = r_eigenv(g_net_ii, g_net_jj)
        
distt_wrong1_train0 = np.zeros((len(graphs_nx_wrong1),len(graphs_nx_train_0)))
WDist_wrong1_train0 = np.zeros((len(graphs_nx_wrong1),len(graphs_nx_train_0)))

for ii, g_net_ii in enumerate(graphs_nx_wrong1):
    for jj, g_net_jj in enumerate(graphs_nx_train_0):
        distt_wrong1_train0[ii,jj], WDist_wrong1_train0[ii,jj] = r_eigenv(g_net_ii, g_net_jj)
        
#ax = sns.displot(distt_wrong1_train1.flatten())

In [ ]:
ax2 = sns.displot(WDist_wrong1_correct1.flatten(), kind = 'kde')

In [ ]:
ax2 = sns.displot(WDist_wrong1_train1.flatten(), kind = 'kde')

In [ ]:
ax2 = sns.displot(WDist_wrong1_train0.flatten(), kind = 'kde')

In [ ]:
distt_wrong0_correct0 = np.zeros((len(graphs_nx_wrong0),len(graphs_nx_correct0)))
WDist_wrong0_correct0 = np.zeros((len(graphs_nx_wrong0),len(graphs_nx_correct0)))

for ii, g_net_ii in enumerate(graphs_nx_wrong0):
    for jj, g_net_jj in enumerate(graphs_nx_correct0):
        distt_wrong0_correct0[ii,jj], WDist_wrong0_correct0[ii,jj] = r_eigenv(g_net_ii, g_net_jj)

distt_wrong0_train0 = np.zeros((len(graphs_nx_wrong0),len(graphs_nx_train_0)))
WDist_wrong0_train0 = np.zeros((len(graphs_nx_wrong0),len(graphs_nx_train_0)))

for ii, g_net_ii in enumerate(graphs_nx_wrong0):
    for jj, g_net_jj in enumerate(graphs_nx_train_0):
        distt_wrong0_train0[ii,jj], WDist_wrong0_train0[ii,jj] = r_eigenv(g_net_ii, g_net_jj)
        
distt_wrong0_train1 = np.zeros((len(graphs_nx_wrong0),len(graphs_nx_train_1)))
WDist_wrong0_train1 = np.zeros((len(graphs_nx_wrong0),len(graphs_nx_train_1)))

for ii, g_net_ii in enumerate(graphs_nx_wrong0):
    for jj, g_net_jj in enumerate(graphs_nx_train_1):
        distt_wrong0_train1[ii,jj], WDist_wrong0_train1[ii,jj] = r_eigenv(g_net_ii, g_net_jj)

In [ ]:
ax2 = sns.displot(WDist_wrong0_correct0.flatten(), kind = 'kde')

In [ ]:
ax2 = sns.displot(WDist_wrong0_train0.flatten(), kind = 'kde')

In [ ]:
ax2 = sns.displot(WDist_wrong0_train1.flatten(), kind = 'kde')

In [ ]:
if 0:
    distt_correct0_train0 = np.zeros((len(graphs_nx_correct0),len(graphs_nx_train_0)))
    WDist_correct0_train0 = np.zeros((len(graphs_nx_correct0),len(graphs_nx_train_0)))

    for ii, g_net_ii in enumerate(graphs_nx_correct0):
        for jj, g_net_jj in enumerate(graphs_nx_train_0):
            distt_correct0_train0[ii,jj], WDist_correct0_train0[ii,jj] = r_eigenv(g_net_ii, g_net_jj)
        
    distt_correct0_train1 = np.zeros((len(graphs_nx_correct0),len(graphs_nx_train_1)))
    WDist_correct0_train1 = np.zeros((len(graphs_nx_correct0),len(graphs_nx_train_1)))

    for ii, g_net_ii in enumerate(graphs_nx_correct0):
        for jj, g_net_jj in enumerate(graphs_nx_train_1):
            distt_correct0_train1[ii,jj], WDist_correct0_train1[ii,jj] = r_eigenv(g_net_ii, g_net_jj)

In [ ]:
if 0:
    distt_correct1_train0 = np.zeros((len(graphs_nx_correct1),len(graphs_nx_train_0)))
    WDist_correct1_train0 = np.zeros((len(graphs_nx_correct1),len(graphs_nx_train_0)))

    for ii, g_net_ii in enumerate(graphs_nx_correct1):
        for jj, g_net_jj in enumerate(graphs_nx_train_0):
            distt_correct1_train0[ii,jj], WDist_correct1_train0[ii,jj] = r_eigenv(g_net_ii, g_net_jj)
        
    distt_correct1_train1 = np.zeros((len(graphs_nx_correct1),len(graphs_nx_train_1)))
    WDist_correct1_train1 = np.zeros((len(graphs_nx_correct1),len(graphs_nx_train_1)))

    for ii, g_net_ii in enumerate(graphs_nx_correct1):
        for jj, g_net_jj in enumerate(graphs_nx_train_1):
            distt_correct1_train1[ii,jj], WDist_correct1_train1[ii,jj] = r_eigenv(g_net_ii, g_net_jj)

In [ ]:
def Wasserstein_Dist(XX, YY):
  
    import numpy as np
    nx = len(XX)
    ny = len(YY)
    n = nx + ny

    XY = np.concatenate([XX,YY])
    X2 = np.concatenate([np.repeat(1/nx, nx), np.repeat(0, ny)])
    Y2 = np.concatenate([np.repeat(0, nx), np.repeat(1/ny, ny)])

    S_Ind = np.argsort(XY)
    XY_Sorted = XY[S_Ind]
    X2_Sorted = X2[S_Ind]
    Y2_Sorted = Y2[S_Ind]

    Res = 0
    E_CDF = 0
    F_CDF = 0
    power = 1

    for ii in range(0, n-2):
        E_CDF = E_CDF + X2_Sorted[ii]
        F_CDF = F_CDF + Y2_Sorted[ii]
        height = abs(F_CDF-E_CDF)
        width = XY_Sorted[ii+1] - XY_Sorted[ii]
        Res = Res + (height ** power) * width;  
 
    return Res

def  Wasserstein_Dist_PVal(XX, YY):
    # Information about Bootstrap: https://towardsdatascience.com/an-introduction-to-the-bootstrap-method-58bcb51b4d60
    import random
    nboots = 10
    WD = Wasserstein_Dist(XX,YY)
    na = len(XX)
    nb = len(YY)
    n = na + nb
    comb = np.concatenate([XX,YY])
    reps = 0
    bigger = 0
    for ii in range(1, nboots):
        e = random.sample(range(n), na)
        f = random.sample(range(n), nb)
        boost_WD = Wasserstein_Dist(comb[e],comb[f]);
        if (boost_WD > WD):
            bigger = 1 + bigger
            
    pVal = bigger/nboots;

    return pVal, WD

In [ ]:
#pVal, WD = Wasserstein_Dist_PVal(WDist_wrong0_train0.flatten(), WDist_wrong0_train1.flatten())
#print(pVal, WD)

In [ ]:
#pVal, WD = Wasserstein_Dist_PVal(WDist_correct0_train0.flatten(), WDist_correct0_train1.flatten())
#print(pVal, WD)

In [ ]:
#pVal, WD = Wasserstein_Dist_PVal(WDist_wrong1_train1.flatten(), WDist_wrong1_train0.flatten())
#print(pVal, WD)

<a id = "dis"></a>
## Discussion
It seems that the current idea is not successful and we should do more investigation. We can also consider about model-specific SafeML.